<a href="https://colab.research.google.com/github/fastdatascience/clinical_trial_risk/blob/fixes_nov_2022/train/ctgov/TrainNumCountriesClassifier_02_Naive_Bayes_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import os
import sys
import pickle as pkl
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import json



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_annotations = pd.read_csv("/content/drive/MyDrive/data/all_annotations.csv")


# Get data

In [ ]:
with open("/content/drive/MyDrive/data/protocols.pkl.gz", "rb") as f:
    file_to_pages = pkl.load(f)

In [ ]:
texts = []

for i in range(len(df_annotations)):
    file_name = df_annotations.file.iloc[i]
    
    pages= file_to_pages[file_name]

    texts.append(" ".join(pages))
df_annotations["text"] = texts

In [8]:
def get_num_countries(x):
  if x is not None and type(x) is str:
    return len(x.split(","))
  return None

In [10]:
df_annotations["num_countries"] = df_annotations.country.apply(get_num_countries)

In [11]:
df_annotations["num_countries"].value_counts()

1.0     7998
2.0      726
3.0      341
4.0      263
5.0      230
6.0      225
7.0      193
8.0      176
10.0     155
9.0      137
11.0     135
12.0     116
13.0     109
14.0      92
15.0      90
17.0      78
18.0      68
16.0      65
21.0      57
20.0      56
19.0      54
23.0      47
22.0      44
24.0      35
25.0      31
26.0      27
29.0      22
30.0      21
27.0      20
31.0      19
28.0      18
32.0      17
33.0      14
41.0      11
34.0      11
38.0       8
39.0       8
37.0       7
43.0       7
36.0       6
35.0       6
44.0       6
40.0       6
48.0       6
47.0       3
42.0       2
49.0       2
50.0       2
45.0       1
58.0       1
46.0       1
62.0       1
51.0       1
54.0       1
Name: num_countries, dtype: int64

In [12]:
def get_num_countries_clean(num):
    if num is None or pd.isna(num):
      return None
    if num >= 10:
      return 10
    return num
df_annotations["num_countries_clean"] = df_annotations["num_countries"].apply(get_num_countries_clean)

# Train and evaluate the number of countries extractor

In [13]:
df_train = df_annotations[(df_annotations.train_val == "train") & (~df_annotations.num_countries_clean.isna())]

In [14]:
df_val = df_annotations[df_annotations.train_val == "val"]

In [15]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, auc, roc_curve
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

In [19]:
NUM_FEATURES = 1500

stops = set(stopwords.words('english')).union(set(stopwords.words('french')))
stops.remove("both")
stops.remove("all")

# Specially engineered regex to include 95%, 95%ci, etc
vectoriser = CountVectorizer(lowercase=True, stop_words=stops, min_df=5, max_features=NUM_FEATURES,
                             token_pattern=r'[59][05]%?(?:ci)?|[a-z][a-z]+')
transformer = TfidfTransformer()

nb = MultinomialNB()
model = make_pipeline(vectoriser, transformer, nb)

In [21]:
model.fit(df_train.text, df_train.num_countries_clean)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['taient', 'tais', 'tait', 'tant', 'tante', 'tantes', 'tants', 'tiez', 'tions'] not in stop_words.
  % sorted(inconsistent)


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=1500, min_df=5,
                                 stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ai', 'aie',
                                             'aient', 'aies', 'ain', 'ait',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'au',
                                             'aura', 'aurai', 'auraient',
                                             'aurais', 'aurait', 'auras',
                                             'aurez', 'auriez', ...},
                                 token_pattern='[59][05]%?(?:ci)?|[a-z][a-z]+')),
                ('tfidftransformer', TfidfTransformer()),
                ('multinomialnb', MultinomialNB())])

In [22]:
df_val["pred_num_countries"] = model.predict(df_val.text)

In [32]:
pred_num_countries = df_val["pred_num_countries"]

In [25]:
num_countries_ground_truths = df_val.num_countries_clean

In [33]:
acc = accuracy_score(num_countries_ground_truths.apply(str), pred_num_countries.apply(str))
print (f"Num countries accuracy {acc}")

Num countries accuracy 0.6986175115207374


''

In [34]:
ConfusionMatrixDisplay.from_predictions(num_countries_ground_truths.apply(str), pred_num_countries.apply(str))
plt.xticks(rotation=90)
;

In [35]:
fake_document = " ".join(vectoriser.vocabulary_)
vectorised_document = vectoriser.transform([fake_document])
transformed_document = transformer.transform(vectorised_document)
probas = np.zeros((transformed_document.shape[1]))

for prediction_idx in range(5):
    print(f"Strongest predictors for class {prediction_idx}\n")
    for i in range(transformed_document.shape[1]):
        zeros = np.zeros(transformed_document.shape)
        zeros[0, i] = transformed_document[0, i]
        proba = nb.predict_log_proba(zeros)
        probas[i] = proba[0, prediction_idx]

    for ctr, j in enumerate(np.argsort(-probas)):
        for w, i in vectoriser.vocabulary_.items():
            if i == j:
                print(f"{ctr}\t{w}")

Streaming output truncated to the last 5000 lines.
1006	multiple
1007	abuse
1008	addition
1009	meet
1010	assessed
1011	replacement
1012	within
1013	reporting
1014	administration
1015	syndrome
1016	kg
1017	rating
1018	may
1019	mediated
1020	true
1021	90
1022	cause
1023	body
1024	clinical
1025	document
1026	drawn
1027	due
1028	safe
1029	daily
1030	elevated
1031	inhibition
1032	phase
1033	either
1034	information
1035	continuous
1036	obtained
1037	nature
1038	take
1039	alone
1040	http
1041	all
1042	blood
1043	direct
1044	followed
1045	code
1046	phone
1047	order
1048	analysis
1049	report
1050	database
1051	devices
1052	mm
1053	numbers
1054	life
1055	informed
1056	estimate
1057	negative
1058	progression
1059	excluded
1060	groups
1061	renal
1062	provide
1063	model
1064	primary
1065	etc
1066	positive
1067	trial
1068	results
1069	routine
1070	although
1071	important
1072	investigators
1073	quality
1074	review
1075	markers
1076	study
1077	dependent
1078	future
1079	therapeutic
1080	duration
1081